In [1]:
import time
from pynq.overlays.base import BaseOverlay
import socket

base = BaseOverlay("base.bit")
btns = base.btns_gpio
leds = base.leds

# Sockets

This notebook has both a client and a server functionality. One PYNQ board in the group will be the client and SENDS the message. Another PYNQ board will be the server and RECEIVES the message.

## Server

Here, we'll build the server code to LISTEN for a message from a specific PYNQ board.

When we send/receive messages, we need to pieces of information which will tell us where to send the information. First, we need the IP address of our friend. Second, we need to chose a port to listen on. For an analogy, Alice expects her friend, Bob, to deliver a package to our back door. With this information, ALICE (server ip address) can wait at the BACK DOOR (port) for BOB (client ip address) to deliver the package.

Format of the information
 ipv4 address: ###.###.###.### (no need for leading zeros if the number is less than three digits)
 port: ##### (it could be 4 or 5 digits long, but must be >1024)
 
Use the socket documentation (Section 18.1.3) to find the appropriate functions https://python.readthedocs.io/en/latest/library/socket.html


In [ ]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# TODO:
# 1: Bind the socket to the pynq board <CLIENT-IP> at port <LISTENING-PORT>
CLIENT_IP = "192.168.0.207"
LISTENING_PORT = 8888

sock.bind((CLIENT_IP, LISTENING_PORT))

# 2: Accept connections
sock.listen(1)

conn, addr = sock.accept()
print(f"Accepted connection from {addr}")

# 3: Receive bytes from the connection
# 4: Print the received message
while True:
    data = conn.recv(1024)

    if not data:
        print("Client disconneted")
        break
    print("Received message: ", data.decode('utf8'))

conn.close()

## Client

Now, we can implement the CLIENT code. 

Back to the analogy, now we're interested in delivering a package to our friend's back door. This means BOB (client ip address) is delivering a package to ALICE (server ip address) at her BACK DOOR (port)

**Remember to start the server before running the client code**

In [9]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# TODO: 
# 1: Connect the socket (sock) to the <SERVER-IP> and choosen port <LISTENING-PORT>
SERVER_IP = '192.168.0.207'
LISTENING_PORT = 8888
sock.connect((SERVER_IP, LISTENING_PORT))

# 2: Send the message "Hello world!\n"
'''message = "Hello world!\n"
sock.sendall(message.encode('utf-8'))'''
# 2a: send messages before closing
for i in range(5):
    message = input("Your message?: ")
    sock.sendall(message.encode('utf-8'))

# 3: Close the socket
sock.close()

Your message?: hello
Your message?: my name is..
Your message?: i don't know...
Your message?: i need to sleep
Your message?: whattttttt


On your server, you should see the message and then the server will shutdown! When we close a socket, both the client and the server are disconnected from the port.

Instead, change the function above to send 5 messages before closing.

The pseudocode looks like this

* connect the socket
* for i in range(5)
    * msg = input("Message to send: ")
    * send the message (msg)
* close the socket